In [1]:
from pwn import *

# get the target hash
# conn = remote("34.124.157.94", 32521)
conn = process(["python", "smart.py"])
conn.recvuntil(b"Target: ")
target = safeeval.expr(conn.recvline().decode().strip())
print(f"{target=}")

[x] Starting local process '/home/sy/miniconda3/bin/python'
[+] Starting local process '/home/sy/miniconda3/bin/python': pid 41041
target=(954, 13, 506, 838)


In [2]:
from numba import njit

p = 1657; q = 1663; r = 1049; s = 1193
@njit() # speedup!
def f(v):
    a = (5 * v[0] + 3 * v[1] + 7 * v[2] + 4 * v[3] + v[0] * v[1] + 10 * (v[2] * v[3])**2) % p
    b = (9 * v[0] + 2 * v[1] + 1 * v[2] + 1 * v[3] + v[1] * v[2] + 11 * (v[0] * v[3])**2) % q
    c = (6 * v[0] + 7 * v[1] + 3 * v[2] + 9 * v[3] + v[2] * v[3] + 12 * (v[0] * v[1])**2) % r
    d = (8 * v[0] + 5 * v[1] + 2 * v[2] + 7 * v[3] + v[3] * v[0] + 13 * (v[1] * v[2])**2) % s
    return (a,b,c,d)

def h(v):
    k = 5555 + ((v[0] * v[1] * v[2] * v[3]) % 3000000)
    for _ in range(k):
        v = f(v)
    return v

In [3]:
from tqdm.auto import tqdm

# find cycle length
ptr = target
found = False
for cycle_len in tqdm(range(1, 3_000_000)):
    ptr = f(ptr)
    if ptr == target:
        print(f"Found cycle length {cycle_len}")
        found = True
        break
assert found

# brute force for any inside the cycle with correct rep count
ptr = target
for offset in tqdm(range(1, cycle_len)):
    ptr = f(ptr)
    n_steps_left = cycle_len - offset
    n_steps_given = 5555 + ((ptr[0] * ptr[1] * ptr[2] * ptr[3]) % 3000000)
    if n_steps_left == n_steps_given:
        print(f"Found solution {ptr}")
        break
assert h(ptr) == target

  0%|          | 0/2999999 [00:00<?, ?it/s]

Found cycle length 1022119


  0%|          | 0/1022118 [00:00<?, ?it/s]

Found solution (396, 976, 233, 6)


In [4]:
# claim flag

conn.sendline(",".join(str(i) for i in ptr))
print(conn.recvall().decode())

[x] Receiving all data
[x] Receiving all data: 54B


/tmp/ipykernel_41026/4026641460.py:3: BytesWarning: Text is not bytes; assuming ASCII, no guarantees. See https://docs.pwntools.com/#bytes
  conn.sendline(",".join(str(i) for i in ptr))


[x] Receiving all data: 113B
[+] Receiving all data: Done (113B)
[*] Process '/home/sy/miniconda3/bin/python' stopped with exit code 0 (pid 41041)
Insert the secret key (4 integers separated by comma)
Welcome admin! grey{hello_smart_admin_;D_hRkPxgxcMB7Yxk4e}

